In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F
import sys
import torch
import torch.nn as nn
import scipy.stats
import torch.optim as optim
from torch.distributions import Categorical
import random as rand
%matplotlib inline

In [ ]:
#Our network
class Policy_Net(nn.Module):
    def __init__(self):
        super(Policy_Net,self).__init__()
        self.input_layer = nn.Linear(env.observation_space.shape[0],28)
        
        self.action_layer = nn.Linear(28,env.action_space.n)
        self.Value_layer = nn.Linear(28,1)
        
        self.ln_proba = []
        self.state_val = []
        self.reward_vec = []
        
    def forward(self,state):
        state = self.input_layer(state)
        state = F.dropout(state, p =.3)
        state = F.relu(state)
        
        state_value = self.Value_layer(state)

     
        self.prob_d = F.softmax(self.action_layer(state), dim =1)
        self.m = Categorical(self.prob_d)
        self.action = self.m.sample()
        
        ## m.log_prob takes the natural log of the probability associated with action
        self.ln_proba.append(self.m.log_prob(self.action))
        ## approximate the Value of a given state
        self.state_val.append(state_value)

        

        return self.action


In [ ]:
device = torch.device('cpu')
Gamma = 0.99
## to reproduce graphs, leave the seed_value 
seed_value = 57899
 
#seed_value = rand.randint(1,100000)

learning_rate=.003
torch.manual_seed(seed_value)
plot_vec = []
plot_vec2 = []
env = gym.make("LunarLander-v2")
#env = gym.make("CartPole-v1")
env.seed(seed_value)
policy = Policy_Net()
policy.to(device)
optimizer = optim.Adam(policy.parameters(), lr=learning_rate)
max_episodes = sys.maxsize
See_results_after_traning = True
plotting_average_after_x_datapoints = 100
plot_vec = []
entropy_vec = []
entropy_vec2 = []
entropy_plot = []



## starts from 1 to infinity
for episodes in range(max_episodes):
    state = env.reset()
    ep_reward = 0
    ## start an episode
    for t in range(1, 10000):
        
        
        
        state = torch.from_numpy(state).float().unsqueeze(0)
        state = state.to(device)
        ## feed a state to the neural netowkr
        action = policy(state)
        ## get entropy for a a state's probability dist.
        entropy_vec.append(scipy.stats.entropy(policy.prob_d.detach().numpy()[0]))
        ## take an action
        action = action.item()
        state, reward, done, _ = env.step(action)       
        policy.reward_vec.append(reward)
        ## accumulate reward  for graphs later
        ep_reward += reward
        if done:                    
            plot_vec.append(ep_reward)
            break
            
    # take the average entropy of the last game    
    entropy_vec2= sum(entropy_vec)/len(entropy_vec)
    
    ## after every 100 episodes, give statistics
    if len(plot_vec) == plotting_average_after_x_datapoints:
        plot_vec2.append(np.average(plot_vec))

        print("Learning rate =",learning_rate,"Seed Value=", seed_value,"Game #",episodes+1,"Entropy =", np.round(entropy_vec2,3),"Length of game=",
          len(policy.reward_vec),"Total Reward =", np.round(plot_vec2[-1],3))

        entropy_plot.append(entropy_vec2)
        
        ## every 500 games, plot a graph of ep_reward and entropy
        if len(entropy_plot)%5 ==0:
            
        
            fig = plt.figure()
            fig.suptitle('Reward')
            plt.plot(plot_vec2)
            plt.show()
            print("Learning rate =",learning_rate,"Seed Value=", seed_value,"Game #",episodes+1,"Entropy =", np.round(entropy_vec2,3),"Length of game=",
          len(policy.reward_vec),"Total Reward =", np.round(plot_vec2[-1],3))

            fig = plt.figure()
            plt.plot(entropy_plot)
            
            fig.suptitle('Entropy')
            plt.show()

            
            plt.close('all')
            
        

        plot_vec=[]
  
        
    ## finish episode
    R = 0
    policy_loss = []
    reward = []
    value_losses = []
    for r , value in zip(reversed(policy.reward_vec), reversed(policy.state_val)):
        ## G_t  - Q(s,a)
        R = r+(Gamma) *R - value 
        reward.insert(0,R)
   
        
    reward = torch.tensor(reward)
    ## standarize the rewards
    reward = (reward - reward.mean()) / (reward.std())
    
    loss = 0
    
    for ln_prob, reward, value in zip(policy.ln_proba,reward, policy.state_val):    
        ## compute the loss function, J(theta)
        action_loss = -ln_prob*reward       
        loss+=action_loss 
    
    optimizer.zero_grad()
    ## back propgate
    loss.backward()
    optimizer.step()
    
    
    policy.reward_vec = []
    policy.state_val = []
    policy.ln_proba = []
    entropy_vec = []
    

## we are done , let's play
    

        
            


#torch.save(policy.state_dict(), 'model_8')







In [ ]:
##At anytime, you could stop training and run this cell to see current result
while True:
    state = env.reset()
    ep_reward = 0
    
    while True:
        
        state = torch.from_numpy(state).float().unsqueeze(0)
        state.to(device)
        probs = policy(state)
        
        action = probs.item()

        state, reward, done, _ = env.step(action)
        ep_reward+=reward
        env.render()
        
        if done:
            print(ep_reward)
            ep_reward=0
            break
        